In [1]:
%matplotlib inline


# Acquiring and analizing AIA cutouts from active regions

*This notebook was modified from the file "downloading_cutouts.ipynb" provided in the example https://tinyurl.com/27afnuuf *

This notebook first downloads a full-disk AIA image given the timestamp and wavelength. Subsequently, it creates a cutout of the active region to be analyzed by specifying the coordinates. Once the appropriate cutout is procured, a histrogram is constructed and its moments calculated.

Date: Jan 2022

In [2]:
import os

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import skew, kurtosis

import astropy.time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import ImageNormalize, SqrtStretch

import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a

ModuleNotFoundError: No module named 'zeep'

First, query a full frame AIA image. Specify the date and wavelength...



In [ ]:
ts = '2017-09-06T11:53:00' #Time in string format
wl = 171 # Wavelength in Angstrom

t0 = astropy.time.Time(ts, scale='utc', format='isot')
q = Fido.search(
    a.Instrument.aia,
    a.Physobs.intensity,
    a.Wavelength(wl*u.angstrom),
    a.Time(t0, t0 + 13*u.s),
)
m = sunpy.map.Map(Fido.fetch(q))

Next, we will create a submap from this image. We will crop the field of view to an active region by specifying the coordinates (in arcsec) of the lower-left and top-right corners.



In [ ]:
ll_x = 100 # Lower-Left, x coord
ll_y = -100
tr_x = 600
tr_y = 400
m_cutout = m.submap(
    SkyCoord(ll_x*u.arcsec, ll_y*u.arcsec, frame=m.coordinate_frame),
    top_right=SkyCoord(tr_x*u.arcsec, tr_y*u.arcsec, frame=m.coordinate_frame),
)
m_cutout.peek()

Before continuing, let's make few checks:
1) Make sure no the timestamp is closer to the flare's start time but no saturated pixel is present.
2) Ensure the cutout encloses the entire active region. You might need to adjust the coordinates.
3) 

Let us visualize the histogram of the AIA intenisty...


In [ ]:
m_cutout


Let's look at the statistics. We can extract some values of interest directly from the data array:



In [ ]:
print("Mean =",m_cutout.data.mean())
print("Median =",np.median(m_cutout.data))
print("Standard Deviation =",m_cutout.data.std())
print("Maximum =",m_cutout.data.max())
print("Minimum =",m_cutout.data.min())
print("Skewness =",skew(m_cutout.data.flatten(),axis=0))
print("Kurtosis =",kurtosis(m_cutout.data.flatten(),axis=0))
print("Integrated = %2.2e"%(np.sum(m_cutout.data*0.60*0.60)))

The units of all quantities but the integrated intensity, can be assumed to be [AIA Intensity].

